## Gömülü Temsiller

Önceki örneğimizde, `vocab_size` uzunluğunda yüksek boyutlu kelime torbası vektörleri üzerinde çalıştık ve düşük boyutlu konumsal temsil vektörlerini açıkça seyrek tekil bir temsil haline dönüştürdük. Bu tekil temsil bellek açısından verimli değildir. Ayrıca, her kelime birbirinden bağımsız olarak ele alındığından, tekil kodlanmış vektörler kelimeler arasındaki anlamsal benzerlikleri ifade etmez.

Bu bölümde, **News AG** veri kümesini keşfetmeye devam edeceğiz. Başlamak için, verileri yükleyelim ve önceki bölümden bazı tanımları alalım.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Gömme Nedir?

**Gömme** fikri, kelimeleri, kelimenin anlamsal anlamını yansıtan daha düşük boyutlu yoğun vektörlerle temsil etmektir. Daha sonra anlamlı kelime gömmeleri nasıl oluşturulacağını tartışacağız, ancak şimdilik gömmeleri bir kelime vektörünün boyutunu azaltmanın bir yolu olarak düşünebiliriz.

Bir gömme katmanı, bir kelimeyi girdi olarak alır ve belirtilen `embedding_size` boyutunda bir çıktı vektörü üretir. Bir anlamda, bir `Dense` katmanına çok benzer, ancak bir giriş olarak tek-sıcak kodlanmış bir vektör almak yerine, bir kelime numarasını alabilir.

Ağımızdaki ilk katman olarak bir gömme katmanı kullanarak, kelime torbasından **gömme torbası** modeline geçebiliriz. Bu modelde, önce metnimizdeki her kelimeyi karşılık gelen gömmeye dönüştürürüz ve ardından bu gömmelerin tümü üzerinde `sum`, `average` veya `max` gibi bir toplama fonksiyonu hesaplarız.

![Beş sıralı kelime için bir gömme sınıflandırıcısını gösteren görsel.](../../../../../translated_images/tr/embedding-classifier-example.b77f021a7ee67eee.webp)

Sınıflandırıcı sinir ağımız şu katmanlardan oluşur:

* `TextVectorization` katmanı: Bu katman bir dizeyi girdi olarak alır ve bir dizi token numarası üretir. Makul bir `vocab_size` (kelime dağarcığı boyutu) belirleyeceğiz ve daha az kullanılan kelimeleri göz ardı edeceğiz. Girdi şekli 1 olacak ve çıktı şekli $n$ olacak, çünkü sonuç olarak $n$ token elde edeceğiz ve her biri 0 ile `vocab_size` arasında sayılar içerecek.
* `Embedding` katmanı: Bu katman $n$ sayıyı alır ve her bir sayıyı belirli bir uzunlukta (örneğimizde 100) yoğun bir vektöre indirger. Böylece, $n$ şekline sahip giriş tensörü, $n\times 100$ şekline sahip bir tensöre dönüştürülür.
* Toplama katmanı: Bu katman, bu tensörün birinci ekseni boyunca ortalamasını alır, yani farklı kelimelere karşılık gelen tüm $n$ giriş tensörlerinin ortalamasını hesaplar. Bu katmanı uygulamak için bir `Lambda` katmanı kullanacağız ve içine ortalamayı hesaplayan fonksiyonu geçireceğiz. Çıktı, 100 boyutunda olacak ve tüm giriş dizisinin sayısal temsili olacaktır.
* Son `Dense` doğrusal sınıflandırıcı.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` çıktısında, **output shape** sütununda, ilk tensör boyutu olan `None` minibatch boyutunu ifade ederken, ikinci boyut ise token dizisinin uzunluğunu ifade eder. Minibatch içindeki tüm token dizilerinin uzunlukları farklıdır. Bununla nasıl başa çıkılacağını bir sonraki bölümde ele alacağız.

Şimdi ağı eğitelim:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Not** verilerin bir alt kümesine dayalı olarak vektörleştirici oluşturduğumuzu unutmayın. Bu, süreci hızlandırmak için yapılır ve metnimizdeki tüm tokenlerin kelime dağarcığında bulunmadığı bir durumla sonuçlanabilir. Bu durumda, bu tokenler göz ardı edilir, bu da biraz daha düşük doğruluğa yol açabilir. Ancak, gerçek hayatta metnin bir alt kümesi genellikle iyi bir kelime dağarcığı tahmini sağlar.


### Değişken dizi boyutlarıyla başa çıkma

Minibatch'lerde eğitim sürecinin nasıl gerçekleştiğini anlayalım. Yukarıdaki örnekte, giriş tensörünün boyutu 1'dir ve 128 uzunluğunda minibatch'ler kullanıyoruz, bu nedenle tensörün gerçek boyutu $128 \times 1$ olur. Ancak, her cümledeki token sayısı farklıdır. Eğer `TextVectorization` katmanını tek bir girişe uygularsak, döndürülen token sayısı, metnin nasıl tokenleştirildiğine bağlı olarak farklılık gösterir:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Ancak, vektörleştiriciyi birkaç dizilim üzerinde uyguladığımızda, dikdörtgen şekilli bir tensör üretmesi gerekir, bu yüzden kullanılmayan öğeleri PAD tokeni (bizim durumumuzda sıfır) ile doldurur:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Not**: Doldurma miktarını en aza indirmek için, bazı durumlarda veri kümesindeki tüm dizileri artan uzunluk sırasına (veya daha doğrusu, token sayısına) göre sıralamak mantıklı olabilir. Bu, her minibatch'in benzer uzunlukta diziler içermesini sağlayacaktır.


## Anlamsal Gömüler: Word2Vec

Önceki örneğimizde, gömme katmanı kelimeleri vektör temsillerine eşlemeyi öğrendi, ancak bu temsillerin anlamsal bir anlamı yoktu. Benzer kelimelerin veya eşanlamlıların, belirli bir vektör mesafesi (örneğin, Öklid mesafesi) açısından birbirine yakın olan vektörlere karşılık geldiği bir vektör temsili öğrenmek güzel olurdu.

Bunu yapmak için, gömme modelimizi [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) gibi bir teknik kullanarak büyük bir metin koleksiyonu üzerinde önceden eğitmemiz gerekiyor. Bu teknik, kelimelerin dağıtılmış bir temsilini üretmek için kullanılan iki ana mimariye dayanır:

 - **Sürekli çanta modeli** (CBoW), burada modeli çevredeki bağlamdan bir kelimeyi tahmin etmek için eğitiriz. $(W_{-2},W_{-1},W_0,W_1,W_2)$ ngrami verildiğinde, modelin amacı $(W_{-2},W_{-1},W_1,W_2)$'den $W_0$'ı tahmin etmektir.
 - **Sürekli atlama-gramı** (skip-gram), CBoW'un tersidir. Model, mevcut kelimeyi tahmin etmek için çevredeki bağlam kelimelerini kullanır.

CBoW daha hızlıdır, ancak skip-gram daha yavaş olmasına rağmen nadir kelimeleri temsil etmede daha başarılıdır.

![Kelimeyi vektörlere dönüştürmek için hem CBoW hem de Skip-Gram algoritmalarını gösteren bir görsel.](../../../../../translated_images/tr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Google News veri seti üzerinde önceden eğitilmiş Word2Vec gömüsünü denemek için **gensim** kütüphanesini kullanabiliriz. Aşağıda 'neural' kelimesine en benzer kelimeleri buluyoruz.

> **Not:** Kelime vektörlerini ilk kez oluşturduğunuzda, bunları indirmek biraz zaman alabilir!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Kelimeden vektör gömme işlemini de çıkarabiliriz, sınıflandırma modelini eğitmek için kullanılabilir. Gömme işlemi 300 bileşene sahiptir, ancak burada açıklık sağlamak için yalnızca vektörün ilk 20 bileşenini gösteriyoruz:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Semantik gömmelerle ilgili harika şey, vektör kodlamasını semantiklere göre manipüle edebilmenizdir. Örneğin, *kral* ve *kadın* kelimelerine mümkün olduğunca yakın, *adam* kelimesinden ise mümkün olduğunca uzak olan bir kelime bulmamızı isteyebiliriz:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Yukarıdaki örnek bazı iç GenSym sihirlerini kullanıyor, ancak altında yatan mantık aslında oldukça basittir. Gömülü temsillerle ilgili ilginç bir şey, gömülü vektörler üzerinde normal vektör işlemleri gerçekleştirebilmenizdir ve bu, kelime **anlamları** üzerindeki işlemleri yansıtır. Yukarıdaki örnek vektör işlemleri açısından ifade edilebilir: **KING-MAN+WOMAN**'a karşılık gelen vektörü hesaplıyoruz (ilgili kelimelerin vektör temsilleri üzerinde `+` ve `-` işlemleri gerçekleştirilir) ve ardından sözlükteki bu vektöre en yakın kelimeyi buluyoruz:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOT**: *man* ve *woman* vektörlerine küçük katsayılar eklemek zorunda kaldık - bunları kaldırmayı deneyin ve ne olduğunu görün.

En yakın vektörü bulmak için, TensorFlow mekanizmasını kullanarak vektörümüz ile kelime haznesindeki tüm vektörler arasındaki mesafelerden oluşan bir vektör hesaplıyoruz ve ardından `argmin` kullanarak en küçük kelimenin indeksini buluyoruz.


Word2Vec, kelime anlamlarını ifade etmek için harika bir yöntem gibi görünse de, aşağıdakiler de dahil olmak üzere birçok dezavantaja sahiptir:

* Hem CBoW hem de skip-gram modelleri **öngörücü gömme** yöntemleridir ve yalnızca yerel bağlamı dikkate alırlar. Word2Vec, küresel bağlamdan faydalanmaz.
* Word2Vec, kelime **morfolojisini** dikkate almaz, yani kelimenin anlamının kök gibi farklı parçalarına bağlı olabileceği gerçeğini göz önünde bulundurmaz.

**FastText**, ikinci sınırlamayı aşmaya çalışır ve Word2Vec üzerine inşa ederek her kelime için ve kelime içindeki karakter n-gramları için vektör temsilleri öğrenir. Temsillerin değerleri, her eğitim adımında bir vektörde ortalanır. Bu, ön eğitimde çok fazla ek hesaplama gerektirse de, kelime gömmelerinin alt kelime bilgilerini kodlamasını sağlar.

Başka bir yöntem olan **GloVe**, kelime gömmeleri için farklı bir yaklaşım kullanır ve kelime-bağlam matrisinin faktörizasyonuna dayanır. Öncelikle, farklı bağlamlarda kelime oluşumlarının sayısını içeren büyük bir matris oluşturur ve ardından bu matrisi daha düşük boyutlarda, yeniden yapılandırma kaybını en aza indirecek şekilde temsil etmeye çalışır.

Gensim kütüphanesi bu kelime gömmelerini destekler ve yukarıdaki model yükleme kodunu değiştirerek bunları deneyebilirsiniz.


## Keras'ta önceden eğitilmiş gömüleri kullanma

Yukarıdaki örneği, gömme katmanımızdaki matrisi Word2Vec gibi anlamsal gömülerle önceden dolduracak şekilde değiştirebiliriz. Önceden eğitilmiş gömme ve metin korpusunun kelime dağarcıkları muhtemelen eşleşmeyecektir, bu yüzden birini seçmemiz gerekir. Burada iki olası seçeneği inceliyoruz: tokenizer kelime dağarcığını kullanmak ve Word2Vec gömme kelime dağarcığını kullanmak.

### Tokenizer kelime dağarcığını kullanma

Tokenizer kelime dağarcığını kullandığımızda, kelime dağarcığındaki bazı kelimelerin karşılık gelen Word2Vec gömüleri olacak, bazıları ise eksik olacaktır. Kelime dağarcığı boyutumuz `vocab_size` ve Word2Vec gömme vektör uzunluğu `embed_size` olduğunda, gömme katmanı `vocab_size`$\times$`embed_size` şeklinde bir ağırlık matrisi ile temsil edilecektir. Bu matrisi kelime dağarcığını gözden geçirerek dolduracağız:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Word2Vec kelime haznesinde bulunmayan kelimeler için, ya sıfır olarak bırakabiliriz ya da rastgele bir vektör oluşturabiliriz.

Şimdi, önceden eğitilmiş ağırlıklarla bir gömme katmanı tanımlayabiliriz:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Not**: `Embedding` oluşturulurken `trainable=False` olarak ayarlandığını fark edin, bu da Embedding katmanını yeniden eğitmediğimiz anlamına gelir. Bu, doğruluğun biraz daha düşük olmasına neden olabilir, ancak eğitimi hızlandırır.

### Gömülü kelime dağarcığını kullanma

Önceki yaklaşımın bir sorunu, TextVectorization ve Embedding'de kullanılan kelime dağarcıklarının farklı olmasıdır. Bu sorunu aşmak için aşağıdaki çözümlerden birini kullanabiliriz:
* Word2Vec modelini kendi kelime dağarcığımızda yeniden eğitmek.
* Veri setimizi, önceden eğitilmiş Word2Vec modelinin kelime dağarcığıyla yüklemek. Veri setini yüklerken kullanılan kelime dağarcıkları yükleme sırasında belirtilebilir.

İkinci yaklaşım daha kolay görünüyor, bu yüzden bunu uygulayalım. Öncelikle, Word2Vec gömülü kelimelerinden alınan belirtilmiş kelime dağarcığı ile bir `TextVectorization` katmanı oluşturacağız:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Gensim kelime gömme kütüphanesi, sizin için otomatik olarak ilgili Keras gömme katmanını oluşturacak olan kullanışlı bir `get_keras_embeddings` fonksiyonunu içerir.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Veri setimizdeki bazı kelimelerin önceden eğitilmiş GloVe kelime hazinesinde eksik olması nedeniyle daha yüksek doğruluk görmememizin nedenlerinden biri budur ve bu nedenle bu kelimeler esasen göz ardı edilir. Bunu aşmak için, veri setimize dayalı kendi gömme vektörlerimizi eğitebiliriz.


## Bağlamsal Gömüler

Word2Vec gibi geleneksel önceden eğitilmiş gömü temsillerinin temel sınırlamalarından biri, bir kelimenin anlamını bir dereceye kadar yakalayabilmelerine rağmen, farklı anlamları ayırt edememeleridir. Bu durum, sonraki modellerde sorunlara yol açabilir.

Örneğin, 'play' kelimesi şu iki cümlede farklı anlamlara sahiptir:
- Tiyatroda bir **oyuna** gittim.
- John arkadaşlarıyla **oynamak** istiyor.

Bahsettiğimiz önceden eğitilmiş gömüler, 'play' kelimesinin her iki anlamını da aynı gömüde temsil eder. Bu sınırlamayı aşmak için, **dil modeli** temelinde gömüler oluşturmamız gerekir. Bu dil modeli, büyük bir metin korpusu üzerinde eğitilmiştir ve kelimelerin farklı bağlamlarda nasıl bir araya gelebileceğini *bilir*. Bağlamsal gömüler konusunu bu eğitimin kapsamı dışında bırakıyoruz, ancak bir sonraki birimde dil modellerinden bahsederken bu konuya geri döneceğiz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
